In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import os
import matplotlib.pyplot as plt
from cv2 import resize
from PIL import Image
from sklearn.metrics import multilabel_confusion_matrix


In [3]:
import tensorflow as tf
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.models import Model

from keras.layers import Concatenate
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.applications import VGG16, VGG19, ResNet50, ResNet101


In [4]:
complete_data_info = pd.read_csv("/kaggle/input/data/Data_Entry_2017.csv")

In [5]:
complete_data_info.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [6]:
data_base_path = "/kaggle/input/data"

In [7]:
complete_data_imgs_paths = list()
imgs_file_names = list()
for single_img_path in Path(data_base_path).glob("images_*/images/*.png"):
    
    complete_data_imgs_paths.append(str(single_img_path))
    imgs_file_names.append(str(single_img_path.parts[-1]))


In [8]:
complete_data_path_info = pd.DataFrame(data={"Image Index":imgs_file_names,"Image Path":complete_data_imgs_paths})

In [9]:
complete_data_path_info.head()

,Image Index,Image Path
0,00006199_010.png,/kaggle/input/data/images_003/images/00006199_...
1,00004833_016.png,/kaggle/input/data/images_003/images/00004833_...
2,00006260_000.png,/kaggle/input/data/images_003/images/00006260_...
3,00004911_010.png,/kaggle/input/data/images_003/images/00004911_...
4,00004186_007.png,/kaggle/input/data/images_003/images/00004186_...


In [10]:
complete_data_all_info = complete_data_info.merge(complete_data_path_info,on = "Image Index",)

In [11]:
complete_data_all_info

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000003_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030801_...
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030802_...
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030803_...
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030804_...


In [12]:
#complete_data_all_info.unique()

In [13]:
complete_data_all_info.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11',
       'Image Path'],
      dtype='object')

In [14]:
complete_data_all_info.drop(complete_data_all_info.columns[2:6],axis = 1,inplace = True)

In [15]:
complete_data_all_info

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Image Path
0,00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,NaN,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,NaN,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,NaN,/kaggle/input/data/images_001/images/00000003_...
...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030801_...
112116,00030802_000.png,No Finding,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030802_...
112117,00030803_000.png,No Finding,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030803_...
112118,00030804_000.png,No Finding,PA,2048,2500,0.168,0.168,NaN,/kaggle/input/data/images_012/images/00030804_...


In [16]:
complete_data_all_info.drop(["Unnamed: 11"],axis=1,inplace=True)

In [17]:
complete_data_all_info.head()

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
0,00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [18]:
complete_data_all_info.columns

Index(['Image Index', 'Finding Labels', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Image Path'],
      dtype='object')

In [19]:
testing_data_info = pd.read_csv("/kaggle/input/data/BBox_List_2017.csv")

In [20]:
testing_data_info

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
979,00029464_015.png,Atelectasis,198.940451,352.900747,615.537778,323.128889,NaN,NaN,NaN
980,00025769_001.png,Atelectasis,701.838229,572.491858,103.537778,63.715556,NaN,NaN,NaN
981,00016837_002.png,Atelectasis,140.913785,658.962969,271.928889,94.435556,NaN,NaN,NaN
982,00020124_003.png,Atelectasis,175.047118,580.456302,244.622222,103.537778,NaN,NaN,NaN


In [21]:
complete_data_all_info.head()

,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
0,00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
1,00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
2,00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
3,00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
4,00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [22]:
complete_data_all_info = complete_data_all_info.set_index("Image Index")

In [23]:
complete_data_all_info.head()

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
Image Index,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [24]:
complete_data_all_info.loc[["00000001_000.png","00000001_001.png"]]

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
Image Index,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...


In [25]:
testing_data_path_info = complete_data_all_info.loc[testing_data_info["Image Index"]]

In [26]:
testing_data_path_info

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
Image Index,,,,,,,
00013118_008.png,Atelectasis,PA,2992,2991,0.143000,0.143000,/kaggle/input/data/images_006/images/00013118_...
00014716_007.png,Atelectasis|Effusion|Mass,AP,3056,2544,0.139000,0.139000,/kaggle/input/data/images_007/images/00014716_...
00029817_009.png,Atelectasis,AP,3056,2544,0.139000,0.139000,/kaggle/input/data/images_012/images/00029817_...
00014687_001.png,Atelectasis|Cardiomegaly|Consolidation,AP,2500,2048,0.168000,0.168000,/kaggle/input/data/images_007/images/00014687_...
00017877_001.png,Atelectasis,AP,2500,2048,0.168000,0.168000,/kaggle/input/data/images_008/images/00017877_...
...,...,...,...,...,...,...,...
00029464_015.png,Atelectasis|Effusion|Infiltration,PA,2021,2021,0.194311,0.194311,/kaggle/input/data/images_012/images/00029464_...
00025769_001.png,Atelectasis,AP,3056,2544,0.139000,0.139000,/kaggle/input/data/images_011/images/00025769_...
00016837_002.png,Atelectasis|Effusion,PA,2592,2446,0.143000,0.143000,/kaggle/input/data/images_008/images/00016837_...


In [27]:
testing_data_path_info.describe

<bound method NDFrame.describe of                                           Finding Labels View Position  \
Image Index                                                              
00013118_008.png                             Atelectasis            PA   
00014716_007.png               Atelectasis|Effusion|Mass            AP   
00029817_009.png                             Atelectasis            AP   
00014687_001.png  Atelectasis|Cardiomegaly|Consolidation            AP   
00017877_001.png                             Atelectasis            AP   
...                                                  ...           ...   
00029464_015.png       Atelectasis|Effusion|Infiltration            PA   
00025769_001.png                             Atelectasis            AP   
00016837_002.png                    Atelectasis|Effusion            PA   
00020124_003.png                 Mass|Nodule|Atelectasis            PA   
00026920_000.png                             Atelectasis            AP   

   

In [28]:
testing_data_info[testing_data_info["Image Index"] == "00020124_003.png"]

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
594,00020124_003.png,Mass,361.244444,284.766823,106.951111,135.395556,NaN,NaN,NaN
982,00020124_003.png,Atelectasis,175.047118,580.456302,244.622222,103.537778,NaN,NaN,NaN


In [29]:
testing_data_info[testing_data_info["Image Index"] == "00014687_001.png"]

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
3,00014687_001.png,Atelectasis,726.237288,494.95142,141.016949,55.322034,NaN,NaN,NaN


In [30]:
testing_data_info[testing_data_info["Image Index"] == "00029464_015.png"]

,Image Index,Finding Label,Bbox [x,y,w,h],Unnamed: 6,Unnamed: 7,Unnamed: 8
322,00029464_015.png,Effusion,719.644444,611.309045,191.146667,51.200000,NaN,NaN,NaN
979,00029464_015.png,Atelectasis,198.940451,352.900747,615.537778,323.128889,NaN,NaN,NaN


In [31]:
training_data_all_info = complete_data_all_info.drop(index=testing_data_path_info.index)

In [32]:
#type(testing_data_path["Image Index"])

In [33]:
#row = complete_data_all_info.loc

In [34]:
testing_data_info.shape

(984, 9)

In [35]:
training_data_all_info.shape

(111240, 7)

In [36]:
type(testing_data_info.index)

pandas.core.indexes.range.RangeIndex

In [37]:
complete_data_all_info.drop(index=list(testing_data_path_info.index),axis=0,inplace=True)


In [38]:
training_data_all_info.head()

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
Image Index,,,,,,,
00000001_000.png,Cardiomegaly,PA,2682,2749,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_001.png,Cardiomegaly|Emphysema,PA,2894,2729,0.143,0.143,/kaggle/input/data/images_001/images/00000001_...
00000001_002.png,Cardiomegaly|Effusion,PA,2500,2048,0.168,0.168,/kaggle/input/data/images_001/images/00000001_...
00000002_000.png,No Finding,PA,2500,2048,0.171,0.171,/kaggle/input/data/images_001/images/00000002_...
00000003_000.png,Hernia,PA,2582,2991,0.143,0.143,/kaggle/input/data/images_001/images/00000003_...


In [39]:
complete_data_all_info.shape

(111240, 7)

In [40]:
testing_data_info = testing_data_info.set_index("Image Index")

In [41]:
testing_data_path_info

,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
Image Index,,,,,,,
00013118_008.png,Atelectasis,PA,2992,2991,0.143000,0.143000,/kaggle/input/data/images_006/images/00013118_...
00014716_007.png,Atelectasis|Effusion|Mass,AP,3056,2544,0.139000,0.139000,/kaggle/input/data/images_007/images/00014716_...
00029817_009.png,Atelectasis,AP,3056,2544,0.139000,0.139000,/kaggle/input/data/images_012/images/00029817_...
00014687_001.png,Atelectasis|Cardiomegaly|Consolidation,AP,2500,2048,0.168000,0.168000,/kaggle/input/data/images_007/images/00014687_...
00017877_001.png,Atelectasis,AP,2500,2048,0.168000,0.168000,/kaggle/input/data/images_008/images/00017877_...
...,...,...,...,...,...,...,...
00029464_015.png,Atelectasis|Effusion|Infiltration,PA,2021,2021,0.194311,0.194311,/kaggle/input/data/images_012/images/00029464_...
00025769_001.png,Atelectasis,AP,3056,2544,0.139000,0.139000,/kaggle/input/data/images_011/images/00025769_...
00016837_002.png,Atelectasis|Effusion,PA,2592,2446,0.143000,0.143000,/kaggle/input/data/images_008/images/00016837_...


In [42]:
#testing_data_info[testing_data_info["Image Index"] == "00020124_003.png"]

In [43]:
testing_data_path_info.reset_index(inplace=True)

In [44]:
#testing_data_info = testing_data_info.set_index("Image Index")

In [45]:
#testing_data_all_info.columns

In [46]:
testing_data_info.shape

(984, 8)

In [47]:
#testing_data_all_info.drop(labels=testing_data_all_info.columns[6:9],axis=1,inplace=True)

In [48]:
testing_data_path_info = testing_data_path_info.reset_index()

In [49]:
testing_data_info = testing_data_info.reset_index()

In [50]:
testing_data_all_info = pd.concat([testing_data_info,testing_data_path_info],axis=1)

In [51]:
testing_data_all_info.shape

(984, 18)

In [52]:
testing_data_all_info.drop(labels=testing_data_all_info.columns[6:9],axis=1,
                          inplace=True)

In [53]:
testing_data_all_info.head()

,Image Index,Finding Label,Bbox [x,y,w,h],index,Image Index,Finding Labels,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Image Path
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,0,00013118_008.png,Atelectasis,PA,2992,2991,0.143,0.143,/kaggle/input/data/images_006/images/00013118_...
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,1,00014716_007.png,Atelectasis|Effusion|Mass,AP,3056,2544,0.139,0.139,/kaggle/input/data/images_007/images/00014716_...
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,2,00029817_009.png,Atelectasis,AP,3056,2544,0.139,0.139,/kaggle/input/data/images_012/images/00029817_...
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,3,00014687_001.png,Atelectasis|Cardiomegaly|Consolidation,AP,2500,2048,0.168,0.168,/kaggle/input/data/images_007/images/00014687_...
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,4,00017877_001.png,Atelectasis,AP,2500,2048,0.168,0.168,/kaggle/input/data/images_008/images/00017877_...


In [54]:
training_data_all_info["Finding Labels"] = training_data_all_info["Finding Labels"].map(lambda x: x.split("|"))
testing_data_all_info["Finding Labels"] = testing_data_all_info["Finding Labels"].map(lambda x: x.split("|"))

In [55]:
merged_list = list()

for single_list in training_data_all_info["Finding Labels"]:
    
    merged_list = merged_list + single_list

unique_diseases = set(merged_list)
unique_diseases = list(unique_diseases)
unique_diseases.remove("No Finding")
disease2idx = dict(zip(unique_diseases,range(0,len(unique_diseases))))

In [56]:
disease2idx

{'Cardiomegaly': 0,
 'Edema': 1,
 'Nodule': 2,
 'Pneumothorax': 3,
 'Mass': 4,
 'Fibrosis': 5,
 'Infiltration': 6,
 'Consolidation': 7,
 'Emphysema': 8,
 'Pleural_Thickening': 9,
 'Atelectasis': 10,
 'Pneumonia': 11,
 'Effusion': 12,
 'Hernia': 13}

In [57]:
def map_diseases(disease_list):
    
    all_zeros = np.zeros(len(disease2idx),)
    
    for single_disease in disease_list:
        
        if single_disease != "No Finding":
        
            all_zeros[disease2idx[single_disease]] = 1
        
    return all_zeros

In [58]:
with ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:
    
    multi_hot_encoded_Y_train = np.array(list(pool.map(map_diseases,training_data_all_info["Finding Labels"])))
    multi_hot_encoded_Y_test = np.array(list(pool.map(map_diseases,testing_data_all_info["Finding Labels"])))

In [59]:
training_data_all_info.reset_index(inplace=True)

In [60]:
multi_hot_encoded_Y_train[0:2]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [61]:
training_data_all_info["Finding Labels"].iloc[0:2]

0               [Cardiomegaly]
1    [Cardiomegaly, Emphysema]
Name: Finding Labels, dtype: object

In [62]:
testing_data_all_info["Finding Labels"].iloc[0:2]

0                    [Atelectasis]
1    [Atelectasis, Effusion, Mass]
Name: Finding Labels, dtype: object

In [63]:
multi_hot_encoded_Y_test[0:2]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.]])

In [64]:
training_data_all_info.to_csv("training_data.csv",index=False)
testing_data_all_info.to_csv("testing_data.csv",index=False)

In [65]:
def custom_training_data_generator(training_data_all_info,multi_hot_encoded_Y_train,mb_size):
    
    idx=training_data_all_info.index
    np.random.shuffle(idx)
    training_data_all_info = training_data_all_info.iloc[idx]
    multi_hot_encoded_Y_train = multi_hot_encoded_Y_train[idx]
    
    for time_step in range(training_data_all_info.shape[0]//mb_size):
        
        X_train_mb = list()
        
        for single_img_path in training_data_all_info.iloc[time_step*mb_size:(time_step+1)+mb_size]["Image Path"]:
            
            resized_single_img = cv2.resize(plt.imread(single_img_path),(1024,1024))
            
            X_train_mb.append(resized_single_img)
            
        X_train_mb = np.array(X_train_mb)
        Y_train_mb = multi_hot_encoded_Y_train[time_step*mb_size:(time_step+1)+mb_size]
        
        yield X_train_mb,Y_train_mb
    

In [66]:
#for X_train_mb,Y_train_mb in custom_training_data_generator(training_data_all_info,multi_hot_encoded_Y_train,40):


In [67]:
from keras.applications import VGG16,VGG19,ResNet50,ResNet101

In [68]:
pretrainned_vgg16_conv_base = VGG16(include_top=False,input_shape=(1024,1024,3),classes=14)
pretrainned_vgg19_conv_base = VGG19(include_top=False,input_shape=(1024,1024,3),classes=14)
pretrainned_ResNet50_conv_base = ResNet50(include_top=False,input_shape=(1024,1024,3),classes=14)
pretrainned_ResNet101_conv_base = ResNet101(include_top=False,input_shape=(1024,1024,3),classes=14)

In [69]:
#pretrainned_ResNet101_conv_base.summary()

In [70]:
class GlobalLsePooling2D(tf.keras.layers.Layer):
    
    def __init__(self,r_hyper_param):
        super(GlobalLsePooling2D,self).__init__()
        self.r = r_hyper_param
        
    def call(self,concatenated_input):
        
        x_star_per_channel = GlobalMaxPooling2D(keepdims = True)(concatenated_input)
        shifted_pix_values = tf.math.exp(self.r*(concatenated_input - x_star_per_channel))
        avged_output = GlobalAveragePooling2D(keepdims = True)(shifted_pix_values)
        logged_output = (1/self.r)*tf.math.log(avged_output)
        layer_output = x_star_per_channel + logged_output
        
        return layer_output

In [71]:
random_output = tf.random.normal(shape = (40,32,32,5120))
our_lse_layer = GlobalLsePooling2D(r_hyper_param=0.9)(random_output)
our_lse_layer.shape

TensorShape([40, 1, 1, 5120])

In [72]:
#  aditya
# def chest_x_ray_cnn():
    
#     input_to_cnn=Input(shape=(1024,1024,3))
#     vgg16_out=pretrained_vgg16_conv_base(input_to_cnn)
#     vgg19_out=pretrained_vgg19_conv_base(input_to_cnn)
#     resnet50_out=pretrained_resnet50_conv_base(input_to_cnn)
#     resnet101_out=pretrained_resnet101_conv_base(input_to_cnn)
    
#     concatenated_output=Concatenate()([vgg16_out,vgg19_out,resnet50_out,resnet101_out])
    
#     pooled_output=GlobalLSEPooling2D(r_hyper_param=0.9)(concatenated_output)
    
#     flattened_output=Flatten()(pooled_output)
    
#     cnn_out=Dense(units=14,activation="sigmoid")(flattened_output)
    
#     return cnn_out

In [76]:
def chest_x_ray_cnn():
    
    input_to_cnn = Input(shape=(1024,1024,3))
    
    pretrained_vgg16_conv_base = VGG16(include_top=False,
                                   input_shape=(1024,1024,3))
    pretrained_vgg19_conv_base = VGG19(include_top=False,
                                  input_shape=(1024,1024,3))
    pretrained_resnet50_conv_base = ResNet50(include_top=False,
                                        input_shape=(1024,1024,3))
    pretrained_resnet101_conv_base = ResNet101(include_top=False,
                                          input_shape=(1024,1024,3))

    pretrained_vgg16_conv_base.trainable = False
    pretrained_vgg19_conv_base.trainable = False
    pretrained_resnet50_conv_base.trainable = False
    pretrained_resnet101_conv_base.trainable = False
    
    vgg16_out = pretrained_vgg16_conv_base(input_to_cnn)
    vgg19_out = pretrained_vgg19_conv_base(input_to_cnn)
    resnet50_out = pretrained_resnet50_conv_base(input_to_cnn)
    resnet101_out = pretrained_resnet101_conv_base(input_to_cnn)
    
    concatenated_output = Concatenate([vgg16_out,vgg19_out,resnet50_out,resnet101_out])
    
    pooled_output = GlobalLsePooling2D(r_hyper_param=0.9)(concatenated_output)
    flattend_output = Flatten()(pooled_output)
    
    cnn_out = Dense(units = 14,activation = "sigmoid")(flattend_output)
    
    return Model(inputs=input_to_cnn,outputs=cnn_out)

In [77]:
our_custom_cnn = chest_x_ray_cnn()

TypeError: Exception encountered when calling layer 'global_lse_pooling2d_1' (type GlobalLsePooling2D).

Inputs to a layer should be tensors. Got '<keras.layers.merging.concatenate.Concatenate object at 0x79da10aad6f0>' (of type <class 'keras.layers.merging.concatenate.Concatenate'>) as input for layer 'global_max_pooling2d_1'.

Call arguments received by layer 'global_lse_pooling2d_1' (type GlobalLsePooling2D):
  • concatenated_input=<keras.layers.merging.concatenate.Concatenate object at 0x79da10aad6f0>

In [ ]:
our_custom_cnn.summary()

In [ ]:
def weighted_bcel(X_train_mb,Y_train_mb):
    cardinality_p = np.count_nonzero(Y_train_mb)
    cardinality_n = (Y_train_mb.shape[0]+Y_train_mb.shape[1])+ cardinality_p
    
    beta_p = (cardinality_p + cardinality_n)/cardinality_p
    beta_n = (cardinality_p + cardinality_n)/cardinality_n
    
    return -tf.reduce_mean(tf.reduce_mean(beta*Y_train_mb*tf.math.log(Y_pred_mb) + beta_n*(1-Y_train_mb)*tf.math.log(1-Y_pred_mb),axis=0))

In [ ]:
train_acc = tf.keras.metrics.Accuracy()
train_precision = tf.keras.metrics.Precision()
train_recall = tf.keras.metrics.Recall()

In [ ]:
def custom_training_data_generator(training_data_all_info, multi_hot_encoded_Y_train, mb_size):
    
    idx = list(training_data_all_info.index)
    np.random.shuffle(idx)
    training_data_all_info = training_data_all_info.iloc[idx]
    multi_hot_encoded_Y_train = multi_hot_encoded_Y_train[idx]
    
    for time_step in range(training_data_all_info.shape[0]//mb_size):
        
        X_train_mb = list()
        
        for single_img_path in training_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size]["Image Path"]:
            
            resized_single_img =  cv2.resize(np.array(Image.open(single_img_path).convert("RGB")),(1024,1024))
            X_train_mb.append(resized_single_img)
            
        X_train_mb = np.array(X_train_mb)
        Y_train_mb = multi_hot_encoded_Y_train[time_step*mb_size:(time_step+1)*mb_size]
        
        yield X_train_mb, Y_train_mb

In [ ]:
def custom_training_data_generator(training_data_all_info,multi_hot_encoded_Y_test,mb_size):
    
    for time_step in range(testing_data_all_info.shape[0]//mb_size):
        
        X_test_mb = list()
        
        for single_img_path in test_data_all_info.iloc[time_step*mb_size:(time_step+1)+mb_size]["Image Path"]:
            
            #resized_single_img = cv2.resize(Image.open(single_img_path.covert("RGB"),(1024,1024))
            resized_single_img =  cv2.resize(np.array(Image.open(single_img_path).convert("RGB")),(1024,1024))
            
            X_train_mb.append(resized_single_img)
                                            
            
        X_train_mb = np.array(X_train_mb)
        Y_train_mb = multi_hot_encoded_Y_train[time_step*mb_size:(time_step+1)+mb_size]
        #bbox_mb = testing_data_all_info.iloc(time_step*mb_size:(time_step+1)+mb_size,[2:6])
        bbox_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,2:6])
        
#         centroid_mb = bbox_mb[:,0,2] + 0.5+np.concatenate(bbox_mb[:,3],bbox_mb[:,2]),axis=1
        centroid_mb = bbox_mb[:,0:2] + 0.5*np.concatenate((bbox_mb[:,3],bbox_mb[:,2]),axis=1)
    
        img_size_mb = np.array(testing_data_all_info.iloc[time_step*mb_size:(time_step+1)*mb_size,8:10])
#         img_size_mb = np.array(testing_data_all_info.iloc(time_step+mb_size:(time_step+1)+mb_size,0:10)
        
        ordered_img_size_mb = np.concatenate((img_size_mb[:,1],img_size_mb[:,0]),axis=1)
        rescaled_centroid_mb = centroid_mb * (32/ordered_centroid_mb)
        
#         ordered_image_size_mb = np.concatenate((img_size_mb[:,1],img_size_mb[:,0],axis=1))                       
#         rescaled_centroid_mb = centroid_mb + (32/ordered_image_size_mb)
                                            
        yield X_train_mb,Y_train_mb
    

In [ ]:
class GlobalLSEPooling2D(tf.keras.layers.Layer):
    
    def __init__(self,r_hyper_param):
        super(GlobalLSEPooling2D, self).__init__()
        self.r = r_hyper_param
        
    def call(self,concatenated_input):
        x_star_per_channel = GlobalMaxPooling2D(keepdims=True)(concatenated_input)
        shifted_pix_values = tf.math.exp(self.r*(concatenated_input - x_star_per_channel))
        avged_output = GlobalAveragePooling2D(keepdims=True)(shifted_pix_values)
        logged_output = (1/self.r)*tf.math.log(avged_output)
        layer_output = x_star_per_channel + logged_output
        
        return layer_output

In [ ]:
def chest_x_ray_cnn():
    
    input_to_cnn = Input(shape=(1024,1024,3))
    
    pretrained_vgg16_conv_base = VGG16(include_top=False,
                                   input_shape=(1024,1024,3))
    pretrained_vgg19_conv_base = VGG19(include_top=False,
                                  input_shape=(1024,1024,3))
    pretrained_resnet50_conv_base = ResNet50(include_top=False,
                                        input_shape=(1024,1024,3))
    pretrained_resnet101_conv_base = ResNet101(include_top=False,
                                          input_shape=(1024,1024,3))

    pretrained_vgg16_conv_base.trainable = False
    pretrained_vgg19_conv_base.trainable = False
    pretrained_resnet50_conv_base.trainable = False
    pretrained_resnet101_conv_base.trainable = False
    
    vgg16_out = pretrained_vgg16_conv_base(input_to_cnn)
    vgg19_out = pretrained_vgg19_conv_base(input_to_cnn)
    resnet50_out = pretrained_resnet50_conv_base(input_to_cnn)
    resnet101_out = pretrained_resnet101_conv_base(input_to_cnn)
    
    concatenated_output = Concatenate()([vgg16_out,vgg19_out,
                                      resnet50_out,resnet101_out])
    
    pooled_output = GlobalLSEPooling2D(r_hyper_param=0.9)(concatenated_output)
    flattened_output = Flatten()(pooled_output)
    
    cnn_out = Dense(units=multi_hot_encoded_Y_train.shape[1],
                    activation="sigmoid")(flattened_output)
    
    return Model(inputs=input_to_cnn,outputs=cnn_out)

In [ ]:
our_custom_cnn = chest_x_ray_cnn()

In [ ]:
def weighted_bcel(Y_train_mb,Y_pred_mb):
    
    cardinality_p = np.count_nonzero(Y_train_mb)
    cardinality_n = (Y_train_mb.shape[0]*Y_train_mb.shape[1]) - cardinality_p
    
    beta_p = (cardinality_p + cardinality_n + 10**(-7))/(cardinality_p + 10**(-7))
    beta_n = (cardinality_p + cardinality_n + 10**(-7))/(cardinality_n + 10**(-7))
    
    return -tf.reduce_mean(tf.reduce_mean(beta_p*Y_train_mb*tf.math.log(Y_pred_mb) \
                          + beta_n*(1-Y_train_mb)*tf.math.log(1-Y_pred_mb),axis=0))

In [ ]:
def predicted_center_pixel_cords(heatmaps):
    
    argmax = list()
    for single_heatmap in heatmaps:
        
        heatmap_channels = cv2.split(single_heatmap)
        argmax_per_heatmap = list()
    
        for single_heatmap_channel in heatmap_channels:    
            single_channel_max_loc = list(np.unravel_index(np.argmax(single_heatmap_channel),single_heatmap_channel.shape))
            argmax_per_heatmap.append(single_channel_max_loc)
        
        argmax.append(np.array(argmax_per_heatmap)/np.array([[7,7]])*224)
        
    return np.array(argmax)

In [ ]:
conv_base = Model(inputs=[our_custom_cnn.input],outputs=[our_custom_cnn.layers[5].output])

In [ ]:
# optimizer = tf.keras.optimizers.RMSprop()

# num_epochs = 25

# for epoch in range(num_epochs):
#     for X_train_mb,Y_train_mb in custom_training_data_generator:
        
#         with tf.GradientTape() as tape:
            
#             Y_pred_mb = our_custom_cnn(X_train_mb)
#             w_bcel_value = weighted_loss(Y_train_mb,Y_pred_mb)
            
#         gradients = tape.gradient(w_bcel_value,our_custom_cnn.trainable_weights)
#         optimizer.apply_gradient(zip(gradients,our_custom_cnn.trainable_weights))
        
#         train_acc.update_state(y_true=Y_train_mb,y_pred = Y_pred_mb)
#         train_precision.update_state(y_true=Y_train_mb,y_pred = Y_pred_mb)
#         train_recall.update_state(y_true=Y_train_mb,y_pred = Y_pred_mb)
        
#         time_steps += 1
        
#         if time_steps % 103 == 0:
#             print("WBCEL Value = {},Time Steps = {}".format(w_bcel_value,time_steps))
#             print("Training Accuracy {},Time Steps ={}".format(train_acc))
#             print("Training Precision {},Time Steps ={}".format(train_acc))
#             print("Training Recall {},Time Steps = {}".format(train_acc))
            
#             train_acc.reset_update()
#             train_precision.reset_update()
#             train_recall.reset_update()
            

In [ ]:
conv_base = Model(inputs=[our_custom_cnn.input],outputs=[our_custom_cnn.layers[5].output])

In [ ]:
optimizer = tf.keras.optimizers.RMSprop()

num_epochs = 25
time_steps = 0
mb_size = 2

for epoch in range(num_epochs):
    for X_train_mb,Y_train_mb in custom_training_data_generator(training_data_all_info,multi_hot_encoded_Y_train,mb_size):
        
        with tf.GradientTape() as tape:
            
            Y_pred_mb = our_custom_cnn(X_train_mb)
            w_bcel_value = weighted_bcel(Y_train_mb,Y_pred_mb)
            
        gradients = tape.gradient(w_bcel_value,our_custom_cnn.trainable_weights)
        optimizer.apply_gradients(zip(gradients,our_custom_cnn.trainable_weights))
        
        train_acc,train_pre,train_rec = compute_performance_metrics(Y_pred_mb,Y_train_mb,0.5)
        
        time_steps += 1
            
        print("\n\nEpoch # {}, Time Step # {}".format(epoch,time_steps))
        print("WBCEL Value = {}, Training Accuracy = {}, Training Precision = {}, Training Recall = {}".format(w_bcel_value,
                                                                                                              train_acc,
                                                                                                              train_pre,
                                                                                                              train_rec))
        conv_base_out = conv_base(X_train_mb).numpy()
        cls_head_params = our_custom_cnn.layers[-1].weights[0].numpy()
        
        heatmaps = np.matmul(conv_base_out,cls_head_params)
        pred_center_pix_loc = predicted_center_pixel_cords(heatmaps)
        
        print("Epoch # {}, Time Step # {}, Predicted Location of Each Disease for Chest X-rays in Mini Batch:".format(epoch,
                                                                                                                     time_steps))
        print(pred_center_pix_loc)

In [ ]:
print("\n\n\n\n")

thresh_probability = np.arange(start=0.3,stop=0.8,step=0.1)
fold_size = 123
K = testing_data_all_info.shape[0]//fold_size

for fold_idx in range(K):
    for p_thresh in thresh_probability:
        
        cv_data = pd.concat([testing_data_all_info[0:(fold_idx*fold_size)],
                             testing_data_all_info[(fold_idx+1)*fold_size:]],axis=0)
        
        testing_data = testing_data_all_info[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        
        multi_hot_encoded_Y_cv = np.concatenate((multi_hot_encoded_Y_test[0:(fold_idx*fold_size)],
                                                multi_hot_encoded_Y_test[(fold_idx+1)*fold_size:]),
                                                axis=0)
        
        multi_hot_encoded_Y_test = multi_hot_encoded_Y_test[(fold_idx*fold_size):(fold_idx+1)*fold_size]
        
        X_cv, Y_cv, rescaled_centroid_cv = custom_testing_data_generator(cv_data,multi_hot_encoded_Y_cv,
                                                                         cv_data.shape[0])
        
        X_test, Y_test, rescaled_centroid_test = custom_testing_data_generator(testing_data,
                                                                               multi_hot_encoded_Y_test,
                                                                               testing_data.shape[0])
        
        Y_pred_cv = our_custom_cnn(X_cv)
        cv_acc,cv_pre,cv_rec = compute_performance_metrics(Y_pred_cv,Y_cv,p_thresh)
        
        cv_conv_base_out = conv_base(X_cv).numpy()
        cv_heatmaps = np.matmul(cv_conv_base_out,cls_head_weights)
        
        Y_pred_test = our_custom_cnn(X_test)
        test_acc,test_pre,test_rec = compute_performance_metrics(Y_pred_test,Y_test,p_thresh)
        
        test_conv_base_out = conv_base(X_test).numpy()
        test_heatmaps = np.matmul(test_conv_base_out,cls_head_weights)
        
        print("Threshold Probability: {}, CV Acc: {}, CV Prec: {}, CV Rec: {}".format(p_thresh,cv_acc,
                                                                                            cv_pre,
                                                                                            cv_rec))
        
        print("Threshold Probability: {}, Test Acc: {}, Test Prec: {}, Test Rec: {}".format(p_thresh,
                                                                                           test_acc,
                                                                                           test_pre,
                                                                                           test_rec))